In [1]:
# this file is intended to try to use pytorchviz to visualize a model
# it isn't too great though...
#
# apt-get update
# apt-get install graphviz
# pip install graphviz
import os
os.chdir("../..")

In [2]:
import torch, torch.nn as nn, torch.nn.functional as F
import pytorch_lightning as pl
import re, random

from models.ranked_transformer import HsqcRankedTransformer
from models.ranked_double_transformer import DoubleTransformer
from datasets.hsqc_folder_dataset import FolderDataModule

In [3]:
from torchviz import make_dot

In [4]:
# model path
folder = "/data/smart4.5/pre_exp_v2/"
path = "j1_both_pair_[09_07_2022_06:23]_[bs=128_epochs=300_lr=1e-05_hsqc_heads=8_hsqc_layers=8_ms_heads=8_ms_layers=8_dropout=0.3_fc_dim=256_hsqc_dim_coords=112,112,32_hsqc_dim_model=256_hsqc_dropout=0_hsqc_ff_dim=1024_hsqc_lr=0.001_hsqc_out_dim=6144]"
chkpt = next((f for f in os.listdir(os.path.join(folder, path, "checkpoints")) if re.search("epoch", f)), None)
full_path = os.path.join(folder, path, "checkpoints", chkpt) if chkpt is not None else None
print(full_path)

/data/smart4.5/pre_exp_v2/j1_both_pair_[09_07_2022_06:23]_[bs=128_epochs=300_lr=1e-05_hsqc_heads=8_hsqc_layers=8_ms_heads=8_ms_layers=8_dropout=0.3_fc_dim=256_hsqc_dim_coords=112,112,32_hsqc_dim_model=256_hsqc_dropout=0_hsqc_ff_dim=1024_hsqc_lr=0.001_hsqc_out_dim=6144]/checkpoints/epoch=299-step=23100.ckpt


In [5]:
model = DoubleTransformer.load_from_checkpoint(full_path).cuda()
model.eval()
my_dir = f"/workspace/smart4.5/tempdata/hyun_fp_data/hsqc_ms_pairs"
dm = FolderDataModule(dir=my_dir, do_hyun_fp=True, input_src=["HSQC", "MS"], batch_size=64)
dm.setup("fit")
val_dl = dm.val_dataloader()

In [7]:
def extract_batch():
    for batch in val_dl:
        hsqc, ms, label = batch
        hsqc, ms, label = hsqc.cuda(), ms.cuda(), label.cuda()
        return hsqc, ms, label

In [8]:
hsqc, ms, label = extract_batch()
model_out = model(hsqc, ms)
out = make_dot(model_out, params=dict(model.named_parameters()))

In [ ]:
out